# Person Re-ID Image Matching

This is for matching two images using an already trained model.

First, import required modules, 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

# Make sure that caffe is on the python path:
caffe_root = '../' 
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

import os



Set Caffe to GPU mode, load the net in the test phase for inference, and configure input preprocessing.

In [28]:
caffe.set_device(0)
caffe.set_mode_gpu()
net = caffe.Net(caffe_root +  'rank_scripts/rank_alexnet/my_unsup_net_deploy.prototxt',
                caffe_root + 'rank_scripts/models17/_iter_250000.caffemodel',
                caffe.TEST)
net1 = caffe.Net(caffe_root +  'rank_scripts/rank_alexnet/my_unsup_net_deploy.prototxt',
                caffe_root + 'rank_scripts/models17/_iter_250000.caffemodel',
                caffe.TEST)

# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_transpose('data', (2,0,1))
#transformer.set_mean('data', np.load(caffe_root + 'rank_scripts/query_256x128_market.npy').mean(1).mean(1)) # mean pixel
transformer.set_raw_scale('data', 255)  # the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_channel_swap('data', (2,1,0))  # the reference model has channels in BGR order instead of RGB

We'll set a batch of 100 to demonstrate batch processing. (Note that the batch size can also be changed on-the-fly.)

In [29]:
# set net to batch size of 50
net.blobs['data'].reshape(50,3,256,128)
net1.blobs['data'].reshape(50,3,256,128)

Feed in the image (with some preprocessing) and use a forward pass.

In [33]:
net.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(caffe_root + 'rank_scripts/images_market/0049_c3s1_004976_00.jpg'))
#check_image =  caffe.io.load_image(caffe_root + 'rank_scripts/images_market/0002_c1s2_050821_02.jpg')
out = net.forward()
vector1=out['fc7'][0]
print vector1
#plt.imshow(check_image)

[-2.01592088 -2.71733451 -2.28586817 ..., -1.60685551 -0.88417357
 -0.12976545]


In [35]:
net1.blobs['data'].data[...] = transformer.preprocess('data', caffe.io.load_image(caffe_root + 'rank_scripts/images_market/0049_c6s1_005651_00.jpg'))
out1 = net1.forward()
vector2=out1['fc7'][0]
print vector2

[-2.38699293 -3.22226834 -2.62830853 ..., -1.55498087 -1.02998424
 -0.2555328 ]


In [36]:
# calculating euclidean distance
sum=0
for i in range(4096):
        diff= vector1[i]-vector2[i]
        diff=diff*diff
        sum=sum+diff
        
print math.sqrt(sum)

16.8446323278
